### Workflow 

1. Use the labeled Dataset class that you wrote for milestone 2 and separate the images in the labeled folder into **train** and **validation** sets (having a validation set is especially important when training on a limited number of samples, since the danger of overfitting to training data is high).
1. Construct and train your supervised melanoma classifier. How does it perform on the test set?
1. Transfer learning is a popular machine learning (ML) approach when training data is scarce. Either choose a pre-trained model out of the ones available in the `torchvision.models` package, or train your own on a (large) dataset of your choice, and fine-tune it on your melanoma data. How does its performance compare to the fully supervised classifier that you trained from scratch in step 2?
1. When transfer learning does not prove to be sufficient, another option that we have is to take advantage of the unlabeled data by employing a semi-supervised ML strategy. Semi-supervised learning is a rapidly developing field, so the best place to look for the state-of-the-art methods is the famous arXiv.org, an open-access archive.  containing thousands of articles across numerous scientific disciplines, including computer science. Learning to work with the arXiv is an important skill for ML engineers involved in solving non trivial problems. **Spend some time to find at least 2-3 different approaches that have shown to be successful in semi-supervised ML for computer vision applications, and list them** along with the arXiv references that you found. It's fine to search arxiv.org and just read abstracts. See
  * https://arxiv.org/search/?query=semi+supervised+image+classification&searchtype=all



### Resources

* https://livebook.manning.com/book/grokking-deep-learning-for-computer-vision/chapter-6
* https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
* https://livebook.manning.com/book/deep-learning-with-pytorch/chapter-2/81
* [Learning Rate Schedulers - Pytorch](https://www.kaggle.com/isbhargav/guide-to-pytorch-learning-rate-scheduling)

##### Convenience

* https://www.youtube.com/watch?v=RKHopFfbPao (try tpdm)

In [22]:
url = 'https://liveproject-resources.s3.amazonaws.com/other/MelanomaDetection.zip'
pth = './data/MelanomaDetection.zip'
unp = './data/MelanomaDetection'
bs  = 24

In [46]:
import torch
import torchvision
from sklearn.model_selection import train_test_split
from torch.nn import BCELoss, BCEWithLogitsLoss, CrossEntropyLoss #not sure which I'd like to use. 
from torch.nn import Linear
from torch.optim import Adam, SGD
from torch.utils.data import Dataset, DataLoader, Subset, SubsetRandomSampler
from torch.optim.lr_scheduler import CyclicLR, OneCycleLR

import numpy as np

In [24]:
%load_ext autoreload
%autoreload 2

from utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
#### Ensure we are running on the GPU

'''
LEFT OFF HERE. GPU is not working

**Update** CUDA now 11.1. cuda.is_available() is True. Need to put the tensors on the GPU. 
**Update** I believe this is fixed. Will remove this comment in future commits. 

* reinstalling torchvision on it's own conda env
* decide if you want to fight this fight or just train on the CPU.
* Relevant links
- https://medium.com/ai%C2%B3-theory-practice-business/use-gpu-in-your-pytorch-code-676a67faed09
- https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html
* Mismatch between torch 1.7 and drivers installed on my machine?
- https://discuss.pytorch.org/t/torch-cuda-is-available-returns-false-nvidia-smi-is-working/20614/2
'''

"\nLEFT OFF HERE. GPU is not working\n\n**Update** CUDA now 11.1. cuda.is_available() is True. Need to put the tensors on the GPU. \n**Update** I believe this is fixed. Will remove this comment in future commits. \n\n* reinstalling torchvision on it's own conda env\n* decide if you want to fight this fight or just train on the CPU.\n* Relevant links\n- https://medium.com/ai%C2%B3-theory-practice-business/use-gpu-in-your-pytorch-code-676a67faed09\n- https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html\n* Mismatch between torch 1.7 and drivers installed on my machine?\n- https://discuss.pytorch.org/t/torch-cuda-is-available-returns-false-nvidia-smi-is-working/20614/2\n"

In [26]:
torch.cuda.is_available()

True

In [27]:
gpu_device = torch.device("cuda:0")

### Create Training and Validation Sets

In [28]:
labeledSet = MILabeled(unp + '/labeled', ToTensor())

sd = 43
cs = len(labeledSet)
ci = range(0, cs)
train_idxs, test_idxs = train_test_split(ci, random_state=sd, train_size=0.75)

train_sampler = SubsetRandomSampler(train_idxs) #subset would work just fine here too I think. 
test_sampler  = SubsetRandomSampler(test_idxs)


In [29]:
labeled_ds = MILabeled(unp + '/labeled', ToTensor())
train_dl = DataLoader(labeled_ds, batch_size=bs, sampler=train_sampler)
test_dl = DataLoader(labeled_ds, batch_size=bs, sampler=test_sampler)

### Construct the Model

In [38]:
mod = torchvision.models.resnet18(False, True)


In [39]:
print(mod)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [40]:
mod.fc

Linear(in_features=512, out_features=1000, bias=True)

In [41]:
#swap out the fully connected layer.  
mod.fc = Linear(512, 2)

In [42]:
mod.to(gpu_device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [43]:
num_epochs=3000

criterion = CrossEntropyLoss() # LogSoftmax + NLLLoss
optimizer = Adam(mod.parameters(), lr=0.1)
scheduler = OneCycleLR(optimizer, max_lr=0.01, steps_per_epoch=len(train_dl), epochs=num_epochs)

### Write a Training Loop

In [44]:

def train_epoch(epoch, model, dataloader, criterion, optimizer, scheduler):
    
    #set train to true
    model.train()
    
    loss_items = []
    
    #loop through batches in data loader
    for bi, (labels, images) in enumerate(dataloader):

        labels = labels.to(gpu_device)
        images = images.to(gpu_device, torch.float)
        
        model.zero_grad()

        pred = model(images)
       
        #https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html
        #https://discuss.pytorch.org/t/torchvision-models-dont-have-softmax-layer/18071/2 (Notes that this is why you use CrossEntropyLoss fn)
        loss = criterion(pred, labels)
        
        loss_items.append(loss.item())
        
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        #print batch results here
        
    #print epoch results here
    print('TRAIN | Epoch Loss: {}'.format(np.mean(loss_items)))



def valid_epoch(epoch, model, dataloader, criterion, bs):
    
    #set train to false
    model.eval()
    
    loss_items = []
    acc_list   = []
    
    #loop through batches in data loader
    for bi, (labels, images) in enumerate(dataloader):
    
        labels = labels.to(gpu_device)
        images = images.to(gpu_device, torch.float)
    
        with torch.no_grad():
            pred = model(images)
    
    
        loss = criterion(pred, labels)
        loss_items.append(loss.item())
        
        #calculate the accuracy
        #https://discuss.pytorch.org/t/torchvision-models-dont-have-softmax-layer/18071/4
        plab = torch.argmax(pred, axis=1) 
        acc  = plab.eq(labels).sum().item() / bs
        
        assert acc >= 0 and acc <= 1.0, "Your accuracy should be between 0 and 1."
        
        #print batch results here
        acc_list.append(acc)
    
    #print epoch results here
    print('VALID | Epoch Loss: {}'.format(np.mean(loss_items)))
    print('VALID | Epoch Accuracy: {}'.format(np.mean(acc_list)))
 




In [45]:

for i in range(num_epochs):
    
    
    train_epoch(i, mod, train_dl, criterion, optimizer, scheduler)
    valid_epoch(i,  mod, test_dl,  criterion, bs)

    


TRAIN | Epoch Loss: 0.6844321148736137
VALID | Epoch Loss: 4.15967591603597
VALID | Epoch Accuracy: 0.3888888888888889
TRAIN | Epoch Loss: 0.4946450335638864
VALID | Epoch Loss: 1.3442583878835042
VALID | Epoch Accuracy: 0.2916666666666667
TRAIN | Epoch Loss: 0.3136783412524632
VALID | Epoch Loss: 0.8328540126482645
VALID | Epoch Accuracy: 0.3611111111111111
TRAIN | Epoch Loss: 0.22751073752130782
VALID | Epoch Loss: 1.5440790057182312
VALID | Epoch Accuracy: 0.4583333333333334
TRAIN | Epoch Loss: 0.2597643584012985
VALID | Epoch Loss: 0.9514079689979553
VALID | Epoch Accuracy: 0.40277777777777773
TRAIN | Epoch Loss: 0.30643530935049057
VALID | Epoch Loss: 0.746489405632019
VALID | Epoch Accuracy: 0.49999999999999994
TRAIN | Epoch Loss: 0.2134608764733587
VALID | Epoch Loss: 1.8501691222190857
VALID | Epoch Accuracy: 0.4444444444444445
TRAIN | Epoch Loss: 0.11486332384603364
VALID | Epoch Loss: 1.096184988816579
VALID | Epoch Accuracy: 0.513888888888889
TRAIN | Epoch Loss: 0.0948403307

TRAIN | Epoch Loss: 0.07029434692646776
VALID | Epoch Loss: 1.6782853603363037
VALID | Epoch Accuracy: 0.47222222222222227
TRAIN | Epoch Loss: 0.0712173195289714
VALID | Epoch Loss: 0.7809181412061056
VALID | Epoch Accuracy: 0.47222222222222227
TRAIN | Epoch Loss: 0.023658161783324822
VALID | Epoch Loss: 0.9047892888387045
VALID | Epoch Accuracy: 0.47222222222222215
TRAIN | Epoch Loss: 0.08994420684341874
VALID | Epoch Loss: 0.8641449809074402
VALID | Epoch Accuracy: 0.5138888888888888
TRAIN | Epoch Loss: 0.07345482760242053
VALID | Epoch Loss: 1.5078746279080708
VALID | Epoch Accuracy: 0.4166666666666667
TRAIN | Epoch Loss: 0.07682917825877666
VALID | Epoch Loss: 1.0425089597702026
VALID | Epoch Accuracy: 0.4444444444444444
TRAIN | Epoch Loss: 0.1575472217186221
VALID | Epoch Loss: 0.8296292622884115
VALID | Epoch Accuracy: 0.47222222222222215
TRAIN | Epoch Loss: 0.2307850598756756
VALID | Epoch Loss: 1.2863454222679138
VALID | Epoch Accuracy: 0.5277777777777778
TRAIN | Epoch Loss: 0.

TRAIN | Epoch Loss: 0.08475980774632522
VALID | Epoch Loss: 0.8076388637224833
VALID | Epoch Accuracy: 0.4861111111111111
TRAIN | Epoch Loss: 0.09517446133707251
VALID | Epoch Loss: 0.8185842335224152
VALID | Epoch Accuracy: 0.5
TRAIN | Epoch Loss: 0.08414617979100772
VALID | Epoch Loss: 1.9209028482437134
VALID | Epoch Accuracy: 0.47222222222222227
TRAIN | Epoch Loss: 0.05024453864565918
VALID | Epoch Loss: 1.1224377155303955
VALID | Epoch Accuracy: 0.4166666666666667
TRAIN | Epoch Loss: 0.10999580698886088
VALID | Epoch Loss: 1.3026479085286458
VALID | Epoch Accuracy: 0.47222222222222227
TRAIN | Epoch Loss: 0.04866392617779119
VALID | Epoch Loss: 0.660971870024999
VALID | Epoch Accuracy: 0.5277777777777778
TRAIN | Epoch Loss: 0.1905797132556992
VALID | Epoch Loss: 1.8090117573738098
VALID | Epoch Accuracy: 0.48611111111111116
TRAIN | Epoch Loss: 0.1120718399594937
VALID | Epoch Loss: 0.958661675453186
VALID | Epoch Accuracy: 0.4583333333333333
TRAIN | Epoch Loss: 0.2543918916157314
V

TRAIN | Epoch Loss: 0.021187554113566875
VALID | Epoch Loss: 1.2078219950199127
VALID | Epoch Accuracy: 0.5416666666666666
TRAIN | Epoch Loss: 0.007515371544286609
VALID | Epoch Loss: 1.560102144877116
VALID | Epoch Accuracy: 0.4583333333333333
TRAIN | Epoch Loss: 0.08005509782898505
VALID | Epoch Loss: 4.044511566559474
VALID | Epoch Accuracy: 0.5
TRAIN | Epoch Loss: 0.017443668197042177
VALID | Epoch Loss: 1.2679737408955891
VALID | Epoch Accuracy: 0.47222222222222215
TRAIN | Epoch Loss: 0.08112829890368241
VALID | Epoch Loss: 3.070959766705831
VALID | Epoch Accuracy: 0.4583333333333333
TRAIN | Epoch Loss: 0.1209319262831871
VALID | Epoch Loss: 1.4310638308525085
VALID | Epoch Accuracy: 0.4861111111111111
TRAIN | Epoch Loss: 0.16933804181670503
VALID | Epoch Loss: 2.33279279867808
VALID | Epoch Accuracy: 0.48611111111111116
TRAIN | Epoch Loss: 0.058696340303868055
VALID | Epoch Loss: 1.246181348959605
VALID | Epoch Accuracy: 0.4861111111111111
TRAIN | Epoch Loss: 0.04563078487158886


TRAIN | Epoch Loss: 0.0008114455725132887
VALID | Epoch Loss: 0.8848889271418253
VALID | Epoch Accuracy: 0.5138888888888888
TRAIN | Epoch Loss: 0.0011098193104511925
VALID | Epoch Loss: 0.8756171862284342
VALID | Epoch Accuracy: 0.5555555555555555
TRAIN | Epoch Loss: 0.0008593428605568729
VALID | Epoch Loss: 0.8954168756802877
VALID | Epoch Accuracy: 0.5416666666666666
TRAIN | Epoch Loss: 0.0014957075763959438
VALID | Epoch Loss: 1.933023989200592
VALID | Epoch Accuracy: 0.5416666666666667
TRAIN | Epoch Loss: 0.0007306756035956953
VALID | Epoch Loss: 0.9746813575426737
VALID | Epoch Accuracy: 0.5277777777777778
TRAIN | Epoch Loss: 0.0007707121819424044
VALID | Epoch Loss: 2.0239574114481607
VALID | Epoch Accuracy: 0.5277777777777778
TRAIN | Epoch Loss: 0.00044413975722688647
VALID | Epoch Loss: 0.9775953491528829
VALID | Epoch Accuracy: 0.5277777777777778
TRAIN | Epoch Loss: 0.0007350345404120162
VALID | Epoch Loss: 1.9950936635335286
VALID | Epoch Accuracy: 0.5277777777777778
TRAIN | 

...

TRAIN | Epoch Loss: 0.0002087766234970851
VALID | Epoch Loss: 3.9403251806894937
VALID | Epoch Accuracy: 0.4583333333333333
TRAIN | Epoch Loss: 6.4338956820263e-05
VALID | Epoch Loss: 2.3735026915868125
VALID | Epoch Accuracy: 0.4444444444444444
TRAIN | Epoch Loss: 6.495770932295792e-05
VALID | Epoch Loss: 2.4299577871958413
VALID | Epoch Accuracy: 0.4444444444444444
TRAIN | Epoch Loss: 4.610561213017458e-05
VALID | Epoch Loss: 3.9290364583333335
VALID | Epoch Accuracy: 0.4444444444444444
TRAIN | Epoch Loss: 0.0002882509059937937
VALID | Epoch Loss: 3.6706788539886475
VALID | Epoch Accuracy: 0.4444444444444444
TRAIN | Epoch Loss: 0.00011140817410445638
VALID | Epoch Loss: 3.9194113413492837
VALID | Epoch Accuracy: 0.4444444444444444
TRAIN | Epoch Loss: 7.816155799186422e-05
VALID | Epoch Loss: 4.114872852961223
VALID | Epoch Accuracy: 0.4444444444444444
TRAIN | Epoch Loss: 3.937596375180874e-05
VALID | Epoch Loss: 3.830937385559082
VALID | Epoch Accuracy: 0.4444444444444445
TRAIN | Epo

TRAIN | Epoch Loss: 0.000200186451495808
VALID | Epoch Loss: 2.3788161277770996
VALID | Epoch Accuracy: 0.4444444444444444
TRAIN | Epoch Loss: 8.617270525844236e-05
VALID | Epoch Loss: 2.429927031199137
VALID | Epoch Accuracy: 0.4444444444444444
TRAIN | Epoch Loss: 9.937797273908342e-05
VALID | Epoch Loss: 2.4287970463434854
VALID | Epoch Accuracy: 0.4444444444444444
TRAIN | Epoch Loss: 5.26280629336335e-05
VALID | Epoch Loss: 2.4838351706663766
VALID | Epoch Accuracy: 0.4444444444444444
TRAIN | Epoch Loss: 4.226111273705361e-05
VALID | Epoch Loss: 2.4065101941426597
VALID | Epoch Accuracy: 0.4444444444444444
TRAIN | Epoch Loss: 3.275133310255894e-05
VALID | Epoch Loss: 2.3952082792917886
VALID | Epoch Accuracy: 0.4444444444444444
TRAIN | Epoch Loss: 7.425035762155727e-05
VALID | Epoch Loss: 2.468100349108378
VALID | Epoch Accuracy: 0.4444444444444444
TRAIN | Epoch Loss: 0.0004755514013855385
VALID | Epoch Loss: 2.325112263361613
VALID | Epoch Accuracy: 0.4444444444444444
TRAIN | Epoch

TRAIN | Epoch Loss: 6.0930138715775684e-05
VALID | Epoch Loss: 4.319112141927083
VALID | Epoch Accuracy: 0.4583333333333334
TRAIN | Epoch Loss: 0.00011778587317426823
VALID | Epoch Loss: 2.3733717997868857
VALID | Epoch Accuracy: 0.45833333333333326
TRAIN | Epoch Loss: 3.578833194036893e-05
VALID | Epoch Loss: 2.4354851245880127
VALID | Epoch Accuracy: 0.4583333333333333
TRAIN | Epoch Loss: 3.47963393453808e-05
VALID | Epoch Loss: 2.5142865975697837
VALID | Epoch Accuracy: 0.4444444444444444
TRAIN | Epoch Loss: 6.71815307993841e-05
VALID | Epoch Loss: 2.5444995164871216
VALID | Epoch Accuracy: 0.4583333333333333
TRAIN | Epoch Loss: 0.00013453193865383843
VALID | Epoch Loss: 2.461486796538035
VALID | Epoch Accuracy: 0.4583333333333333
TRAIN | Epoch Loss: 4.085472645393955e-05
VALID | Epoch Loss: 2.3695077498753867
VALID | Epoch Accuracy: 0.45833333333333326
TRAIN | Epoch Loss: 7.129141288584963e-05
VALID | Epoch Loss: 4.2145281235377
VALID | Epoch Accuracy: 0.4583333333333334
TRAIN | Ep

TRAIN | Epoch Loss: 4.715011216051478e-05
VALID | Epoch Loss: 4.330657402674357
VALID | Epoch Accuracy: 0.4583333333333334
TRAIN | Epoch Loss: 6.14467119573549e-05
VALID | Epoch Loss: 4.116870085398356
VALID | Epoch Accuracy: 0.4583333333333334
TRAIN | Epoch Loss: 7.91393593837191e-05
VALID | Epoch Loss: 3.984361489613851
VALID | Epoch Accuracy: 0.4583333333333333
TRAIN | Epoch Loss: 6.079815765198353e-05
VALID | Epoch Loss: 3.0628799200057983
VALID | Epoch Accuracy: 0.4444444444444445
TRAIN | Epoch Loss: 3.84080974202204e-05
VALID | Epoch Loss: 2.317698359489441
VALID | Epoch Accuracy: 0.4444444444444444
TRAIN | Epoch Loss: 4.085472677875909e-05
VALID | Epoch Loss: 2.4417971769968667
VALID | Epoch Accuracy: 0.4583333333333333
TRAIN | Epoch Loss: 8.951624217193707e-05
VALID | Epoch Loss: 4.717915693918864
VALID | Epoch Accuracy: 0.4583333333333333
TRAIN | Epoch Loss: 0.00010178912089031655
VALID | Epoch Loss: 2.779701312383016
VALID | Epoch Accuracy: 0.47222222222222227
TRAIN | Epoch L

TRAIN | Epoch Loss: 3.508584859446273e-05
VALID | Epoch Loss: 9.778923630714417
VALID | Epoch Accuracy: 0.47222222222222215
TRAIN | Epoch Loss: 0.0003139163823756722
VALID | Epoch Loss: 5.301954905192058
VALID | Epoch Accuracy: 0.4305555555555556
TRAIN | Epoch Loss: 0.00014071805396399993
VALID | Epoch Loss: 2.387499431769053
VALID | Epoch Accuracy: 0.4305555555555556
TRAIN | Epoch Loss: 4.109456364988416e-05
VALID | Epoch Loss: 3.478213826815287
VALID | Epoch Accuracy: 0.4583333333333333
TRAIN | Epoch Loss: 3.319411059367537e-05
VALID | Epoch Loss: 2.4536622365315757
VALID | Epoch Accuracy: 0.4444444444444444
TRAIN | Epoch Loss: 4.087033747574813e-05
VALID | Epoch Loss: 2.4523712793986
VALID | Epoch Accuracy: 0.4583333333333333
TRAIN | Epoch Loss: 7.855466641818307e-05
VALID | Epoch Loss: 2.4569364388783774
VALID | Epoch Accuracy: 0.45833333333333326
TRAIN | Epoch Loss: 0.0008202918940826619
VALID | Epoch Loss: 8.682836532592773
VALID | Epoch Accuracy: 0.47222222222222227
TRAIN | Epoc

In [ ]:
### LEFT OFF HERE. 
'''
- should you make sure you convert labels to floats? Are they Bytes now? 
- try starting with a lower learning rate?
- this never gets a high accuracy. Seems random. Is the problem that it's just moving around the training space too much?
- what errors might have you made in defining your training loop?
- Can you add tqdm
- should you try SGD with OneCycle?  Is there a simpler scheduler? 
- Can you do a learning rate finder in pytorch?
- What if you quickly rewrote this code to use the new version of fast.ai? 
'''


### Perform Transfer Learning

### Testing

In [12]:
dataiter = iter(test_dl)
labels, images = dataiter.next()
print(images.shape)
print(labels)

torch.Size([24, 3, 32, 32])
tensor([0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0])


In [21]:
for bi, b in enumerate(test_dl,0):
    labels, images = b
    print(labels)
    print(bi)
    break

tensor([0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0])
0


In [26]:
#messing with Subset
test_ds  = Subset(labeled_ds, test_idxs)
test_dl2 = DataLoader(test_ds, batch_size=bs)

dataiter = iter(test_dl2)
labels, images = dataiter.next()
print(images.shape)
print(labels)

torch.Size([24, 3, 32, 32])
tensor([0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1])
